In [2]:
import pandas as pd
import numpy as np
import holidays
from xgboost import XGBRegressor

In [17]:
# 处理重复值的核心步骤
def process_df(df):
    """处理重复日期并计算平均值"""
    return (
        df.drop_duplicates(subset=['date_time'])             # 对数值列取平均
        .sort_values('date_time')               # 按日期排序
    )

In [78]:
# 数据准备
traffic_vol = []
cloud = []
rain = []
snow = []
temp = []
weather_a = []
weather_b = []
traffic_vol_val = []
cloud_val = []
rain_val = []
snow_val = []
temp_val = []
weather_a_val = []
weather_b_val = []

for i in range(2019,2025):
    traffic_vol.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/train_data/{i}/traffic_volume.csv', parse_dates=['date_time']))
    cloud.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/train_data/{i}/clouds_all.csv', parse_dates=['date_time']))
    rain.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/train_data/{i}/rain_1h.csv', parse_dates=['date_time']))
    snow.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/train_data/{i}/snow_1h.csv', parse_dates=['date_time']))
    temp.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/train_data/{i}/temp.csv', parse_dates=['date_time']))
    weather_a.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/train_data/{i}/weather_description.csv', parse_dates=['date_time']))
    weather_b.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/train_data/{i}/weather_main.csv', parse_dates=['date_time']))

traffic_vol_val.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/test_input/2025_01/traffic_volume.csv', parse_dates=['date_time']))
cloud_val.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/test_input/2025_01/clouds_all.csv', parse_dates=['date_time']))
rain_val.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/test_input/2025_01/rain_1h.csv', parse_dates=['date_time']))
snow_val.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/test_input/2025_01/snow_1h.csv', parse_dates=['date_time']))
temp_val.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/test_input/2025_01/temp.csv', parse_dates=['date_time']))
weather_a_val.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/test_input/2025_01/weather_description.csv', parse_dates=['date_time']))
weather_b_val.append(pd.read_csv(f'D:/allforwork/AI/nnsp/NS-2025-05-data/test_input/2025_01/weather_main.csv', parse_dates=['date_time']))


In [79]:
for i in range(len(traffic_vol)):
    traffic_vol[i] = process_df(traffic_vol[i])
for i in range(len(cloud)):
    cloud[i] = process_df(cloud[i])
for i in range(len(rain)):
    rain[i] = process_df(rain[i])
for i in range(len(snow)):
    snow[i] = process_df(snow[i])
for i in range(len(temp)):
    temp[i] = process_df(temp[i])
for i in range(len(weather_a)):
    weather_a[i] = process_df(weather_a[i])
for i in range(len(weather_b)):
    weather_b[i] = process_df(weather_b[i])

In [80]:
traffic_vol = pd.concat(traffic_vol, axis=0)
cloud = pd.concat(cloud, axis=0)
rain = pd.concat(rain, axis=0)
snow = pd.concat(snow, axis=0)
temp = pd.concat(temp, axis=0)
weather_a = pd.concat(weather_a, axis=0)
weather_b = pd.concat(weather_b, axis=0)


In [81]:
traffic_vol.head(1)

,date_time,traffic_volume
0,2019-10-02 09:00:00,5545


In [82]:
# 生成完整时间序列（填充缺失日期）
full_index = pd.date_range(
    start='2019-10-02 09:00:00',
    end='2024-12-31 23:00:00',
    freq='H'
)

# 最终结果整理
traffic_vol = (
    traffic_vol.set_index('date_time')
    .reindex(full_index)
    .reset_index()
)
cloud = (
    cloud.set_index('date_time')
    .reindex(full_index)
    .reset_index()

)
rain = (
    rain.set_index('date_time')
    .reindex(full_index)
    .reset_index()

)
snow = (
    snow.set_index('date_time')
    .reindex(full_index)
    .reset_index()

)
temp = (
    temp.set_index('date_time')
    .reindex(full_index)
    .reset_index()

)
weather_a = (
    weather_a.set_index('date_time')
    .reindex(full_index)
    .reset_index()

)
weather_b = (
    weather_b.set_index('date_time')
    .reindex(full_index)
    .reset_index()
)

traffic_vol.rename(columns={'index': 'date_time'}, inplace=True)
cloud.rename(columns={'index': 'date_time'}, inplace=True)
rain.rename(columns={'index': 'date_time'}, inplace=True)
snow.rename(columns={'index': 'date_time'}, inplace=True)
temp.rename(columns={'index': 'date_time'}, inplace=True)
weather_a.rename(columns={'index': 'date_time'}, inplace=True)
weather_b.rename(columns={'index': 'date_time'}, inplace=True)

C:\Users\47328\AppData\Local\Temp\ipykernel_29776\2810628914.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_index = pd.date_range(


In [83]:
traffic_vol.columns

Index(['date_time', 'traffic_volume'], dtype='object')

In [84]:
whole_dataset = traffic_vol.merge(cloud, on='date_time', how='outer')
whole_dataset = whole_dataset.merge(rain, on='date_time', how='outer')
whole_dataset = whole_dataset.merge(snow, on='date_time', how='outer')
whole_dataset = whole_dataset.merge(temp, on='date_time', how='outer')
whole_dataset = whole_dataset.merge(weather_a, on='date_time', how='outer')
whole_dataset = whole_dataset.merge(weather_b, on='date_time', how='outer')
whole_dataset.head()


,date_time,traffic_volume,clouds_all,rain_1h,snow_1h,temp,weather_description,weather_main
0,2019-10-02 09:00:00,5545.0,40.0,0.0,0.0,288.28,scattered clouds,Clouds
1,2019-10-02 10:00:00,4516.0,75.0,0.0,0.0,289.36,broken clouds,Clouds
2,2019-10-02 11:00:00,4767.0,90.0,0.0,0.0,289.58,overcast clouds,Clouds
3,2019-10-02 12:00:00,5026.0,90.0,0.0,0.0,290.13,overcast clouds,Clouds
4,2019-10-02 13:00:00,4918.0,75.0,0.0,0.0,291.14,broken clouds,Clouds


In [88]:
whole_dataset.head(1)

,date_time,traffic_volume,clouds_all,rain_1h,snow_1h,temp,weather_description,weather_main
0,2019-10-02 09:00:00,5545.0,40.0,0.0,0.0,288.28,scattered clouds,Clouds


In [89]:

print(len(traffic_vol), len(cloud), len(snow), len(temp), len(weather_a), len(weather_b))
print(len(whole_dataset))

46023 46023 46023 46023 46023 46023
46023


In [93]:
whole_dataset = whole_dataset.dropna()

In [94]:

print(len(whole_dataset))
# 生成时间特征
def create_features(df):
    df = df.copy()
    if 'date_time' in df.columns:
        df.set_index('date_time', inplace=True)
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['month'] = df.index.month
    df['is_weekend'] = (df.index.dayofweek >= 5).astype(int)
    
    # 中国节假日标记
    cn_holidays = holidays.CountryHoliday('CN')
    df['is_holiday'] = df.index.to_series().apply(
        lambda x: x in cn_holidays).astype(int)
    return df



34030


In [95]:
train_df = create_features(whole_dataset)
print(len(train_df))

34030


In [96]:
# 自定义编码函数（解决类别爆炸问题）
def safe_onehot(df, col_name, max_categories=15):
    """带类别限制的独热编码"""
    # 获取高频类别
    top_cats = df[col_name].value_counts().nlargest(max_categories-1).index.tolist()
    
    # 创建新特征
    encoded = (
        df[col_name]
        .apply(lambda x: x if x in top_cats else 'other')
        .str.get_dummies()
        .add_prefix(f"{col_name}_")
    )

    df = pd.concat([df.drop(col_name, axis=1), encoded], axis=1)

    return df

In [97]:
train_df = safe_onehot(train_df, 'weather_main')
train_df = safe_onehot(train_df, 'weather_description')

In [98]:
train_df.head(5)

,traffic_volume,clouds_all,rain_1h,snow_1h,temp,hour,dayofweek,month,is_weekend,is_holiday,...,weather_description_heavy snow,weather_description_light intensity drizzle,weather_description_light rain,weather_description_light snow,weather_description_mist,weather_description_moderate rain,weather_description_other,weather_description_overcast clouds,weather_description_scattered clouds,weather_description_sky is clear
date_time,,,,,,,,,,,,,,,,,,,,,
2019-10-02 09:00:00,5545.0,40.0,0.0,0.0,288.28,9,2,10,0,1,...,0,0,0,0,0,0,0,0,1,0
2019-10-02 10:00:00,4516.0,75.0,0.0,0.0,289.36,10,2,10,0,1,...,0,0,0,0,0,0,0,0,0,0
2019-10-02 11:00:00,4767.0,90.0,0.0,0.0,289.58,11,2,10,0,1,...,0,0,0,0,0,0,0,1,0,0
2019-10-02 12:00:00,5026.0,90.0,0.0,0.0,290.13,12,2,10,0,1,...,0,0,0,0,0,0,0,1,0,0
2019-10-02 13:00:00,4918.0,75.0,0.0,0.0,291.14,13,2,10,0,1,...,0,0,0,0,0,0,0,0,0,0


In [99]:
print(train_df.columns)

Index(['traffic_volume', 'clouds_all', 'rain_1h', 'snow_1h', 'temp', 'hour',
       'dayofweek', 'month', 'is_weekend', 'is_holiday', 'weather_main_Clear',
       'weather_main_Clouds', 'weather_main_Drizzle', 'weather_main_Fog',
       'weather_main_Haze', 'weather_main_Mist', 'weather_main_Rain',
       'weather_main_Smoke', 'weather_main_Snow', 'weather_main_Squall',
       'weather_main_Thunderstorm', 'weather_description_Sky is Clear',
       'weather_description_broken clouds', 'weather_description_few clouds',
       'weather_description_haze', 'weather_description_heavy intensity rain',
       'weather_description_heavy snow',
       'weather_description_light intensity drizzle',
       'weather_description_light rain', 'weather_description_light snow',
       'weather_description_mist', 'weather_description_moderate rain',
       'weather_description_other', 'weather_description_overcast clouds',
       'weather_description_scattered clouds',
       'weather_description_sky is

In [100]:
cols_name = ['traffic_volume', 'clouds_all', 'rain_1h', 'snow_1h', 'temp', 'hour',
       'dayofweek', 'month', 'is_weekend', 'is_holiday', 'weather_main_Clear',
       'weather_main_Clouds', 'weather_main_Drizzle', 'weather_main_Fog',
       'weather_main_Haze', 'weather_main_Mist', 'weather_main_Rain',
       'weather_main_Smoke', 'weather_main_Snow', 'weather_main_Squall',
       'weather_main_Thunderstorm', 'weather_description_Sky is Clear',
       'weather_description_broken clouds', 'weather_description_few clouds',
       'weather_description_fog', 'weather_description_haze',
       'weather_description_heavy snow',
       'weather_description_light intensity drizzle',
       'weather_description_light rain', 'weather_description_light snow',
       'weather_description_mist', 'weather_description_moderate rain',
       'weather_description_other', 'weather_description_overcast clouds',
       'weather_description_scattered clouds',
       'weather_description_sky is clear']

# 生成滞后特征
for name in cols_name:
    for lag in [1, 2, 3, 24, 24*7, 24*30, 24*90, 24*365]:
        try:
            train_df[f'{name}_lag_{lag}'] = train_df[f'{name}'].shift(lag)
        except KeyError:
            train_df[f'{name}_lag_{lag}'] = np.nan
        


C:\Users\47328\AppData\Local\Temp\ipykernel_29776\237782552.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[f'{name}_lag_{lag}'] = train_df[f'{name}'].shift(lag)
C:\Users\47328\AppData\Local\Temp\ipykernel_29776\237782552.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[f'{name}_lag_{lag}'] = train_df[f'{name}'].shift(lag)
C:\Users\47328\AppData\Local\Temp\ipykernel_29776\237782552.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many time

In [101]:
print(len(train_df))
# 清理缺失值
train_df.dropna(inplace=True)
print(len(train_df))

34030
0


In [41]:

# 拆分特征和标签
X = train_df.drop('traffic_volume', axis=1)
y = train_df['traffic_volume']

# 训练模型
model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1
)
model.fit(X, y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.01, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

In [48]:


# 生成预测时间范围
future_dates = pd.date_range(
    start='2025-01-01 00:00:00',
    end='2025-09-30 23:00:00',
    freq='H'
)


C:\Users\47328\AppData\Local\Temp\ipykernel_17832\184785524.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(


In [64]:

# 递归预测函数
def recursive_forecast(last_known_window, model, future_dates):
    predictions = []
    history = last_known_window.copy()
    
    for date in future_dates:
        # 生成特征
        features = create_features(pd.DataFrame(index=[date]))
        for name in cols_name:
            for lag in [1, 2, 3, 24, 24*7, 24*30, 24*90, 24*365]:
                try:
                    # print(features.columns, f'{name}_lag_{lag}')
                    print(features[features.index.duplicated()])
                    print(history.loc[date - pd.Timedelta(hours=lag), f'{name}'])
                    features[f'{name}_lag_{lag}'] = history.loc[date - pd.Timedelta(hours=lag), f'{name}']
                except KeyError:
                    features[f'{name}_lag_{lag}'] = np.nan
        
        # 预测并存储结果
        pred = model.predict(features)[0]
        predictions.append(pred)
        
        # 更新历史记录
        history.loc[date] = {
            'traffic_volume': pred,
            **features.iloc[0].to_dict()
        }
    
    return pd.Series(predictions, index=future_dates)


In [81]:
test_df = train_df.copy()
test_df.reset_index(inplace=True)
dups = list(test_df.duplicated(subset=['date_time']))
print(dups)

[False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, True, False, True, False, True, False, True, False, True, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, F

In [65]:

# 获取最后可用的历史窗口（包含足够滞后数据）
last_known = train_df.tail(24*365*2)  # 保留最近两周数据

# 执行预测
forecast = recursive_forecast(last_known, model, future_dates)

# 保存结果
forecast.to_csv('D:/allforwork/AI/nnsp/NS-2025-05-data/results.csv', header=['traffic_volume'])

Empty DataFrame
Columns: [hour, dayofweek, month, is_weekend, is_holiday]
Index: []
1580.0
Empty DataFrame
Columns: [hour, dayofweek, month, is_weekend, is_holiday, traffic_volume_lag_1]
Index: []
2041.0
Empty DataFrame
Columns: [hour, dayofweek, month, is_weekend, is_holiday, traffic_volume_lag_1, traffic_volume_lag_2]
Index: []
2483.0
Empty DataFrame
Columns: [hour, dayofweek, month, is_weekend, is_holiday, traffic_volume_lag_1, traffic_volume_lag_2, traffic_volume_lag_3]
Index: []
1000.0
Empty DataFrame
Columns: [hour, dayofweek, month, is_weekend, is_holiday, traffic_volume_lag_1, traffic_volume_lag_2, traffic_volume_lag_3, traffic_volume_lag_24]
Index: []
1092.0
Empty DataFrame
Columns: [hour, dayofweek, month, is_weekend, is_holiday, traffic_volume_lag_1, traffic_volume_lag_2, traffic_volume_lag_3, traffic_volume_lag_24, traffic_volume_lag_168]
Index: []
1479.0
Empty DataFrame
Columns: [hour, dayofweek, month, is_weekend, is_holiday, traffic_volume_lag_1, traffic_volume_lag_2, tr

ValueError: cannot reindex on an axis with duplicate labels